In [1]:
import psycopg2
from faker import Faker
from tabulate import tabulate
from tqdm import tqdm
import random

In [77]:
#conn.close()

conn = psycopg2.connect(
    dbname="vpnbot",
    user="seva",
    password="1234",
    host="172.23.7.191"
)
cur = conn.cursor()
faker = Faker()

In [4]:

cur.execute("""
    SELECT schema_name
    FROM information_schema.schemata
    WHERE schema_name NOT LIKE 'pg_%' AND schema_name != 'information_schema';
""")
schemas = cur.fetchall()

schemas

[('public',), ('vpb',)]

In [5]:
def show_table(table):
    cur.execute(f"""
    SELECT
        column_name,
        data_type,
        is_nullable,
        column_default
    FROM
        information_schema.columns
    WHERE
        table_schema = 'vpb' AND table_name = '{table}';
    """)
    columns = cur.fetchall()

    print(f"Table Description for vpb.\"{table}\":\n")
    print(tabulate(columns, headers=["Column", "Data Type", "Nullable", "Default"], tablefmt="grid"))

show_table("Users")

Table Description for vpb."Users":

+-----------+-------------------+------------+-----------+
| Column    | Data Type         | Nullable   | Default   |
+===========+===================+============+===========+
| user_id   | integer           | NO         |           |
+-----------+-------------------+------------+-----------+
| full_name | character varying | YES        |           |
+-----------+-------------------+------------+-----------+
| login     | character varying | YES        |           |
+-----------+-------------------+------------+-----------+
| balance   | numeric           | NO         |           |
+-----------+-------------------+------------+-----------+


In [30]:
def clear_table(table_name):
        cur.execute(f'DELETE FROM vpb."{table_name}";')
        conn.commit()
# clear_table("Users_Servers")

In [ ]:
def fill_users(num_rows):

    cur.execute('SELECT COALESCE(MAX(user_id), 0) FROM vpb."Users";')
    last_user_id = cur.fetchone()[0]

    insert_query = """
        INSERT INTO vpb."Users" (user_id, full_name, login, balance)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (user_id) DO NOTHING;  -- Avoid duplicate user_ids
    """
    
    for i in tqdm(range(num_rows), desc="Inserting rows"):
        user_id = last_user_id + i + 1
        full_name = f"{faker.name()}"
        login = f"{faker.user_name()}{random.choice(['-', '_','='])}{random.randint(1,1000000)}"
        balance = round(faker.pyfloat(min_value=0, max_value=10000, right_digits=2), 2)
        
        cur.execute(insert_query, (user_id, full_name, login, balance))
    
    conn.commit()
    print(f"\nSuccessfully inserted {num_rows} rows into vpb.\"Users\" table.")

fill_users(30000)

In [28]:
def show_content(table_name):
    cur.execute(f'SELECT * FROM vpb."{table_name}" ORDER BY 1 ASC LIMIT 10;')
    first_10 = cur.fetchall()

    cur.execute(f'SELECT * FROM vpb."{table_name}" ORDER BY 1 DESC LIMIT 10;')
    last_10 = cur.fetchall()

    cur.execute(f'SELECT * FROM vpb."{table_name}" ORDER BY RANDOM() LIMIT 3;')
    random_3 = cur.fetchall()

    col_names = [desc[0] for desc in cur.description]

    print(f"\nFirst 10 Rows from vpb.\"{table_name}\":\n")
    print(tabulate(first_10, headers=col_names, tablefmt="grid"))

    print(f"\nLast 10 Rows from vpb.\"{table_name}\":\n")
    print(tabulate(last_10, headers=col_names, tablefmt="grid"))

    print(f"\n3 Random Rows from vpb.\"{table_name}\":\n")
    print(tabulate(random_3, headers=col_names, tablefmt="grid"))

show_content("Users")


First 10 Rows from vpb."Users":

+-----------+------------------+-----------------------+-----------+
|   user_id | full_name        | login                 |   balance |
+===========+==================+=======================+===========+
|         1 | Stephen Rose     | eringlenn-749094      |   4728.93 |
+-----------+------------------+-----------------------+-----------+
|         2 | Christopher Mata | ellistom=331823       |   1780.21 |
+-----------+------------------+-----------------------+-----------+
|         3 | Robert Carroll   | fcrawford_353198      |   7819.14 |
+-----------+------------------+-----------------------+-----------+
|         4 | James Schmidt    | angelalowe-11565      |   3113.27 |
+-----------+------------------+-----------------------+-----------+
|         5 | Jimmy Murphy     | dharris-186179        |   7328.9  |
+-----------+------------------+-----------------------+-----------+
|         6 | April Brown      | lanethomas=41468      |   7551.17 |


In [48]:
def fill_payments(num_rows):

    cur.execute('SELECT user_id FROM vpb."Users";')
    user_ids = [row[0] for row in cur.fetchall()]
    
    if not user_ids:
        print("No users found in the vpb.\"Users\" table. Please add users before adding transactions.")
        return

    currencies = ['RUB', 'YOO', 'TGX']

    insert_query = """
        INSERT INTO vpb."Payments" (payment_date, amount, currency, user_id)
        VALUES (%s, %s, %s, %s);
    """

    for _ in tqdm(range(num_rows), desc="Inserting transactions"):
        payment_date = faker.date_time_this_year()
        amount = round(random.uniform(1, 1000), 2)
        currency = random.choice(currencies)
        user_id = random.choice(user_ids)

        cur.execute(insert_query, (payment_date, amount, currency, user_id))
    
    conn.commit()
fill_payments(100000)

Inserting transactions:   0%|          | 0/100000 [00:00<?, ?it/s]

Inserting transactions: 100%|██████████| 100000/100000 [00:30<00:00, 3242.30it/s]


In [51]:
show_table("Payments")
show_content("Payments")

Table Description for vpb."Payments":

+--------------+-----------------------------+------------+----------------------------------------------------+
| Column       | Data Type                   | Nullable   | Default                                            |
+==============+=============================+============+====================================================+
| payment_id   | integer                     | NO         | nextval('vpb."Payments_payment_id_seq"'::regclass) |
+--------------+-----------------------------+------------+----------------------------------------------------+
| payment_date | timestamp without time zone | YES        | CURRENT_TIMESTAMP                                  |
+--------------+-----------------------------+------------+----------------------------------------------------+
| currency     | character                   | NO         |                                                    |
+--------------+-----------------------------+-----------

In [52]:
def fill_configurations(num_rows):
    cur.execute('SELECT user_id FROM vpb."Users";')
    user_ids = [row[0] for row in cur.fetchall()]
    
    if not user_ids:
        print("No users found in the vpb.\"Users\" table. Please add users before adding configurations.")
        return

    protocols = ['VLESS', 'WireGuard', 'Outline', 'OpenVPN', 'X-Ray']

    insert_query = """
        INSERT INTO vpb."Configurations" (user_id, protocol, token, expires)
        VALUES (%s, %s, %s, %s);
    """

    for _ in tqdm(range(num_rows), desc="Inserting configurations"):
        user_id = random.choice(user_ids)  
        protocol = random.choice(protocols)
        token = faker.uuid4()  
        expires = faker.date_between(start_date="today", end_date="+1y")  

        cur.execute(insert_query, (user_id, protocol, token, expires))
    
    conn.commit()

fill_configurations(10000)

Inserting configurations: 100%|██████████| 10000/10000 [00:03<00:00, 3297.85it/s]


In [53]:
show_table("Configurations")
show_content("Configurations")

Table Description for vpb."Configurations":

+-----------+-------------------+------------+---------------------------------------------------------+
| Column    | Data Type         | Nullable   | Default                                                 |
+===========+===================+============+=========================================================+
| config_id | integer           | NO         | nextval('vpb."Configurations_config_id_seq"'::regclass) |
+-----------+-------------------+------------+---------------------------------------------------------+
| user_id   | integer           | YES        |                                                         |
+-----------+-------------------+------------+---------------------------------------------------------+
| protocol  | character varying | YES        |                                                         |
+-----------+-------------------+------------+---------------------------------------------------------+
| token   

In [54]:
def fill_servers(num_rows):
    protocols = ['VLESS', 'WireGuard', 'Outline', 'OpenVPN', 'X-Ray']

    insert_query = """
        INSERT INTO vpb."Servers" (server_id, ip_address, protocol)
        VALUES (%s, %s, %s);
    """

    for _ in tqdm(range(num_rows), desc="Inserting servers"):
        server_id = faker.uuid4()
        ip_address = faker.ipv4_public()
        protos = set()
        for _ in range(random.randint(3, 10)):
            protos.add(random.choice(protocols))
        
        protocol = ' '.join([x for x in protos])

        cur.execute(insert_query, (server_id, ip_address, protocol))
    
    conn.commit()

fill_servers(1000)

Inserting servers: 100%|██████████| 1000/1000 [00:00<00:00, 3086.43it/s]


In [55]:
show_table("Servers")
show_content("Servers")

Table Description for vpb."Servers":

+------------+-------------------+------------+-----------+
| Column     | Data Type         | Nullable   | Default   |
+============+===================+============+===========+
| server_id  | character         | NO         |           |
+------------+-------------------+------------+-----------+
| ip_address | character varying | NO         |           |
+------------+-------------------+------------+-----------+
| protocol   | character varying | YES        |           |
+------------+-------------------+------------+-----------+

First 10 Rows from vpb."Servers":

+--------------------------------------+-----------------+---------------------------------------+
| server_id                            | ip_address      | protocol                              |
+======================================+=================+=======================================+
| 00111ad5-45d4-4bd9-affe-f47080d913ee | 156.220.148.133 | OpenVPN X-Ray VLESS           

In [40]:
def fill_users_servers():
    cur.execute("""
        SELECT c.user_id, sc.server_id
        FROM vpb."Configurations" c
        JOIN vpb."Servers_Configs" sc ON c.config_id = sc.config_id;
    """)
    config_links = cur.fetchall()

    if not config_links:
        print("No configurations linked to servers found. Please ensure configurations and server links exist.")
        return

    insert_query = """
        INSERT INTO vpb."Users_Servers" (user_id, server_id)
        VALUES (%s, %s)
    """

    existing_pairs = set()
    for user_id, server_id in tqdm(config_links, desc="Inserting user-server links"):
        pair = (user_id, server_id)
        if pair not in existing_pairs:
            cur.execute(insert_query, pair)
            existing_pairs.add(pair)

    conn.commit()

fill_users_servers()

Inserting user-server links:   0%|          | 0/10000 [00:00<?, ?it/s]

Inserting user-server links: 100%|██████████| 10000/10000 [00:02<00:00, 4157.19it/s]


In [41]:
show_table("Users_Servers")
show_content("Users_Servers")

Table Description for vpb."Users_Servers":

+-----------+-------------+------------+-----------+
| Column    | Data Type   | Nullable   | Default   |
+===========+=============+============+===========+
| user_id   | integer     | YES        |           |
+-----------+-------------+------------+-----------+
| server_id | character   | YES        |           |
+-----------+-------------+------------+-----------+

First 10 Rows from vpb."Users_Servers":

+-----------+--------------------------------------+
|   user_id | server_id                            |
+===========+======================================+
|         4 | 617315de-7c4c-40cb-90d7-13859b47d943 |
+-----------+--------------------------------------+
|         6 | b03cd963-9a61-446d-9393-7874015d7969 |
+-----------+--------------------------------------+
|         6 | 9c9ffbcd-852f-47a0-95c8-f1135c294996 |
+-----------+--------------------------------------+
|        14 | 140a90f3-3b04-4140-a24f-5e6e08787897 |
+-----------+

In [25]:
def fill_server_configs():
    cur.execute('SELECT server_id FROM vpb."Servers";')
    server_ids = [row[0] for row in cur.fetchall()]

    cur.execute("""
        SELECT config_id FROM vpb."Configurations"
        WHERE config_id NOT IN (SELECT config_id FROM vpb."Servers_Configs");
    """)
    config_ids = [row[0] for row in cur.fetchall()]

    if not server_ids:
        print("No servers found in the vpb.\"Server\" table. Please add servers first.")
        return

    if not config_ids:
        print("No configurations found in the vpb.\"Configuration\" table. Please add configurations first.")
        return

    insert_query = """
        INSERT INTO vpb."Servers_Configs" (server_id, config_id)
        VALUES (%s, %s);
    """

    for config_id in tqdm(config_ids, desc="Linking configurations to servers"):
        server_id = random.choice(server_ids)
        cur.execute(insert_query, (server_id, config_id))

    conn.commit()
    
    print(f"\nSuccessfully linked {len(config_ids)} configurations to servers in vpb.\"Server_Config\" table.")
fill_server_configs()

Linking configurations to servers: 100%|██████████| 10000/10000 [00:02<00:00, 3525.56it/s]


Successfully linked 10000 configurations to servers in vpb."Server_Config" table.


In [29]:
show_table("Servers_Configs")
show_content("Servers_Configs")

Table Description for vpb."Servers_Configs":

+-----------+-------------+------------+-----------+
| Column    | Data Type   | Nullable   | Default   |
+===========+=============+============+===========+
| server_id | character   | NO         |           |
+-----------+-------------+------------+-----------+
| config_id | integer     | NO         |           |
+-----------+-------------+------------+-----------+

First 10 Rows from vpb."Servers_Configs":

+--------------------------------------+-------------+
| server_id                            |   config_id |
+======================================+=============+
| 00111ad5-45d4-4bd9-affe-f47080d913ee |        5845 |
+--------------------------------------+-------------+
| 00111ad5-45d4-4bd9-affe-f47080d913ee |        7691 |
+--------------------------------------+-------------+
| 00111ad5-45d4-4bd9-affe-f47080d913ee |        5777 |
+--------------------------------------+-------------+
| 00111ad5-45d4-4bd9-affe-f47080d913ee |    

In [57]:
def full_inspect_user(user_id):
    query = """
    SELECT
        u.user_id,
        u.full_name,
        u.login,
        u.balance,
        p.payment_id,
        p.payment_date,
        p.amount,
        p.currency,
        c.config_id,
        c.protocol AS config_protocol,
        c.token,
        c.expires,
        s.server_id,
        s.ip_address,
        s.protocol AS server_protocol
    FROM
        vpb."Users" u
    LEFT JOIN
        vpb."Payments" p ON u.user_id = p.user_id
    LEFT JOIN
        vpb."Configurations" c ON u.user_id = c.user_id
    LEFT JOIN
        vpb."Servers_Configs" sc ON c.config_id = sc.config_id
    LEFT JOIN
        vpb."Servers" s ON sc.server_id = s.server_id
    WHERE
        u.user_id = %s;
    """

    # Execute the query
    cur.execute(query, (user_id,))
    results = cur.fetchall()

    if not results:
        print(f"No user found with user_id: {user_id}")
        return

    headers = [
        "User ID", "Full Name", "Login", "Balance",
        "Payment ID", "Payment Date", "Amount", "Currency",
        "Config ID", "Config Protocol", "Token", "Expires",
        "Server ID", "IP Address", "Server Protocol"
    ]

    unique_rows = set(results)
    print(f"\nDetails for User ID: {user_id}")
    print(tabulate(unique_rows, headers=headers, tablefmt="grid"))
full_inspect_user(15)


Details for User ID: 15
+-----------+--------------------+--------------------+-----------+--------------+---------------------+----------+------------+-------------+-------------------+--------------------------------------+------------+--------------------------------------+---------------+---------------------------------------+
|   User ID | Full Name          | Login              |   Balance |   Payment ID | Payment Date        |   Amount | Currency   |   Config ID | Config Protocol   | Token                                | Expires    | Server ID                            | IP Address    | Server Protocol                       |
+===========+====================+====================+===========+==============+=====================+==========+============+=============+===================+======================================+============+======================================+===============+=======================================+
|        15 | Allison Burnett MD | larryjosep

In [58]:
def payments_inspect_user(user_id):
    query = """
    SELECT
        u.user_id,
        u.full_name,
        u.login,
        u.balance,
        p.payment_id,
        p.payment_date,
        p.amount,
        p.currency
    FROM
        vpb."Users" u
    LEFT JOIN
        vpb."Payments" p ON u.user_id = p.user_id
    WHERE
        u.user_id = %s;
    """

    cur.execute(query, (user_id,))
    results = cur.fetchall()

    if not results:
        print(f"No user found with user_id: {user_id}")
        return

    headers = [
        "User ID", "Full Name", "Login", "Balance",
        "Payment ID", "Payment Date", "Amount", "Currency"]

    print(f"\nDetails for User ID: {user_id}")
    print(tabulate(results, headers=headers, tablefmt="grid"))

payments_inspect_user(15)


Details for User ID: 15
+-----------+--------------------+--------------------+-----------+--------------+---------------------+----------+------------+
|   User ID | Full Name          | Login              |   Balance |   Payment ID | Payment Date        |   Amount | Currency   |
+===========+====================+====================+===========+==============+=====================+==========+============+
|        15 | Allison Burnett MD | larryjoseph=489509 |   9048.86 |        84043 | 2024-08-29 23:23:19 |   123.05 | TGX        |
+-----------+--------------------+--------------------+-----------+--------------+---------------------+----------+------------+
|        15 | Allison Burnett MD | larryjoseph=489509 |   9048.86 |        89354 | 2024-09-25 21:53:42 |   492.63 | RUB        |
+-----------+--------------------+--------------------+-----------+--------------+---------------------+----------+------------+
|        15 | Allison Burnett MD | larryjoseph=489509 |   9048.86 |     

In [62]:
def servers_inspect_user(user_id):
    query = """
    SELECT
        u.user_id,
        u.full_name,
        u.login,
        s.server_id,
        s.ip_address,
        s.protocol
    FROM
        vpb."Users" u
    JOIN
        vpb."Users_Servers" us ON u.user_id = us.user_id
    JOIN
        vpb."Servers" s ON us.server_id = s.server_id
    WHERE
        u.user_id = %s;
    """

    # Execute the query
    cur.execute(query, (user_id,))
    results = cur.fetchall()

    if not results:
        print(f"No servers found for user_id: {user_id}")
        return

    headers = [
        "User ID", "Full Name", "Login",
        "Server ID", "IP Address", "Protocol"
    ]

    print(f"\nServers for User ID: {user_id}")
    print(tabulate(results, headers=headers, tablefmt="grid"))

servers_inspect_user(15)


Servers for User ID: 15
+-----------+--------------------+--------------------+--------------------------------------+---------------+---------------------------------------+
|   User ID | Full Name          | Login              | Server ID                            | IP Address    | Protocol                              |
+===========+====================+====================+======================================+===============+=======================================+
|        15 | Allison Burnett MD | larryjoseph=489509 | 166897cb-c424-4e67-b01f-a9c1802096e3 | 56.143.93.176 | Outline X-Ray OpenVPN VLESS WireGuard |
+-----------+--------------------+--------------------+--------------------------------------+---------------+---------------------------------------+
|        15 | Allison Burnett MD | larryjoseph=489509 | 7bd72007-5351-41b5-96be-d83714c455cd | 210.60.87.23  | WireGuard OpenVPN Outline VLESS       |
+-----------+--------------------+--------------------+--------------

In [65]:
def configs_inspect_user(user_id):
    query = """
    SELECT
        u.user_id,
        u.full_name,
        u.login,
        c.config_id,
        c.protocol,
        c.token,
        c.expires
    FROM
        vpb."Users" u
    JOIN
        vpb."Configurations" c ON u.user_id = c.user_id
    WHERE
        u.user_id = %s;
    """

    # Execute the query
    cur.execute(query, (user_id,))
    results = cur.fetchall()

    if not results:
        print(f"No configurations found for user_id: {user_id}")
        return

    headers = [
        "User ID", "Full Name", "Login",
        "Config ID", "Protocol", "Token", "Expires"
    ]

    print(f"\nConfigurations for User ID: {user_id}")
    print(tabulate(results, headers=headers, tablefmt="grid"))

configs_inspect_user(15)


Configurations for User ID: 15
+-----------+--------------------+--------------------+-------------+------------+--------------------------------------+------------+
|   User ID | Full Name          | Login              |   Config ID | Protocol   | Token                                | Expires    |
+===========+====================+====================+=============+============+======================================+============+
|        15 | Allison Burnett MD | larryjoseph=489509 |        1187 | Outline    | b83e0487-22d7-40ae-b54e-a6aa7389bb58 | 2025-12-12 |
+-----------+--------------------+--------------------+-------------+------------+--------------------------------------+------------+
|        15 | Allison Burnett MD | larryjoseph=489509 |        2815 | OpenVPN    | 331cc5d4-71eb-439e-862c-27f417066b16 | 2025-05-30 |
+-----------+--------------------+--------------------+-------------+------------+--------------------------------------+------------+


In [75]:
def full_inspect_server(server_id, rows='all'):
    limit_clause = f"LIMIT {rows}" if isinstance(rows, int) else ""
    query_server = f"""
    select * 
    from 
        vpb."Servers" S
    where S.server_id = %s
    """
    cur.execute(query_server, (server_id,))
    server = cur.fetchall()

    print(f"\nServer ID: {server_id}")
    if server:
        print(tabulate(server, headers=["Server ID", "IP Adress", "Protocols"], tablefmt="grid"))
    else:
        print("No users found for this server.")

    query_users = f"""
    SELECT
        u.user_id,
        u.full_name,
        u.login,
        u.balance
    FROM
        vpb."Users_Servers" us
    JOIN
        vpb."Users" u ON us.user_id = u.user_id
    WHERE
        us.server_id = %s
    {limit_clause};
    """
    cur.execute(query_users, (server_id,))
    users = cur.fetchall()

    print(f"\nUsers connected to Server ID: {server_id}")
    if users:
        print(tabulate(users, headers=["User ID", "Full Name", "Login", "Balance"], tablefmt="grid"))
    else:
        print("No users found for this server.")

    query_configs = f"""
    SELECT
        c.config_id,
        c.protocol,
        c.token,
        c.expires
    FROM
        vpb."Servers_Configs" sc
    JOIN
        vpb."Configurations" c ON sc.config_id = c.config_id
    WHERE
        sc.server_id = %s
    {limit_clause};
    """
    cur.execute(query_configs, (server_id,))
    configs = cur.fetchall()

    print(f"\nConfigurations linked to Server ID: {server_id}")
    if configs:
        print(tabulate(configs, headers=["Config ID", "Protocol", "Token", "Expires"], tablefmt="grid"))
    else:
        print("No configurations found for this server.")

full_inspect_server(server_id="166897cb-c424-4e67-b01f-a9c1802096e3", rows =10)


Server ID: 166897cb-c424-4e67-b01f-a9c1802096e3
+--------------------------------------+---------------+---------------------------------------+
| Server ID                            | IP Adress     | Protocols                             |
+======================================+===============+=======================================+
| 166897cb-c424-4e67-b01f-a9c1802096e3 | 56.143.93.176 | Outline X-Ray OpenVPN VLESS WireGuard |
+--------------------------------------+---------------+---------------------------------------+

Users connected to Server ID: 166897cb-c424-4e67-b01f-a9c1802096e3
+-----------+---------------------+-----------------------+-----------+
|   User ID | Full Name           | Login                 |   Balance |
+===========+=====================+=======================+===========+
|     10290 | Justin Ayers        | donna34=431401        |    935.52 |
+-----------+---------------------+-----------------------+-----------+
|        15 | Allison Burnett MD  | la

In [ ]:
from datetime import datetime, timedelta
import statistics

def full_inspect_payments(start_date=None, stop_date=None):
    if not stop_date:
        stop_date = datetime.now()
    if not start_date:
        start_date = stop_date - timedelta(weeks=1)

    start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
    stop_date_str = stop_date.strftime('%Y-%m-%d %H:%M:%S')
    query = """
    SELECT
        p.payment_id,
        p.payment_date,
        p.amount,
        p.currency,
        u.user_id,
        u.full_name,
        u.login,
        u.balance
    FROM
        vpb."Payments" p
    JOIN
        vpb."Users" u ON p.user_id = u.user_id
    WHERE
        p.payment_date BETWEEN %s AND %s
    ORDER BY
        p.payment_date ASC;
    """

    cur.execute(query, (start_date_str, stop_date_str))
    payments = cur.fetchall()

    if not payments:
        print(f"No payments found between {start_date_str} and {stop_date_str}.")
        return

    payment_amounts = [row[2] for row in payments]

    total_earned = sum(payment_amounts)
    total_transactions = len(payment_amounts)
    highest_transaction = max(payment_amounts)
    median_transaction = statistics.median(payment_amounts)

    print(f"\nOverview of Payments Between {start_date_str} and {stop_date_str}")
    print("-" * 60)
    print(f"Total Earned:          {total_earned:.2f}")
    print(f"Total Transactions:    {total_transactions}")
    print(f"Highest Transaction:   {highest_transaction:.2f}")
    print(f"Median Transaction:    {median_transaction:.2f}")
    print("-" * 60)

    headers = [
        "Payment ID", "Payment Date", "Amount", "Currency",
        "User ID", "Full Name", "Login", "User Balance"
    ]

    print("\nDetailed Payments:")
    print(tabulate(payments, headers=headers, tablefmt="grid"))

full_inspect_payments()


Overview of Payments Between 2024-12-10 11:12:26 and 2024-12-17 11:12:26
------------------------------------------------------------
Total Earned:          962059.27
Total Transactions:    1921
Highest Transaction:   999.64
Median Transaction:    493.27
------------------------------------------------------------

Detailed Payments:
+--------------+---------------------+----------+------------+-----------+-------------------------+----------------------------+----------------+
|   Payment ID | Payment Date        |   Amount | Currency   |   User ID | Full Name               | Login                      |   User Balance |
+==============+=====================+==========+============+===========+=========================+============================+================+
|        43999 | 2024-12-10 11:12:42 |    51.94 | YOO        |     29653 | Rachel Rogers           | rollinsangela_537774       |        4722.3  |
+--------------+---------------------+----------+------------+-----------+-